# Gerekli Kütüphanelerin Yüklenmesi


In [ ]:
import pandas as pd
!pip install pytrends
import pytrends
from pytrends.request import TrendReq
pytrend = TrendReq()

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import seaborn as sns
sns.set(color_codes=True)
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot as plt

import plotly.express as px

# Anahter Kelimelerin Belirlenip Eklenmesi

In [ ]:
KEYWORDS=['Masa','Sandalye','Koltuk'] 

# Tarih Aralığı Seçimi

In [ ]:
startd=widgets.DatePicker(
    description='Start',
    disabled=False
)
display(startd)

DatePicker(value=None, description='Start')

In [ ]:
endd=widgets.DatePicker(
    description='End',
    disabled=False
)
display(endd)

DatePicker(value=None, description='End')

In [ ]:
DATE_INTERVAL=str(startd.value) +str(" ") + str(endd.value)

In [ ]:
print(DATE_INTERVAL)

2019-01-01 2021-05-18


# Ülke Seçimi

In [ ]:
countrys=widgets.SelectMultiple(
    options=['TR', 'GB', 'US'], 
    value=['TR'],
    
    description='Countries',
    disabled=False
)
print("Select Multiple Options by clicking Ctrl button")
COUNTRY=countrys.value
display(countrys)


Select Multiple Options by clicking Ctrl button


SelectMultiple(description='Countries', index=(0,), options=('TR', 'GB', 'US'), value=('TR',))

# Kategori ve Arama Türü Düzenlemeleri

In [ ]:
CATEGORY=0 # All Categories Food&Drink, Art

In [ ]:
SEARCH_TYPE='' # '' -> 'web searches', 
               #'images','news','youtube','froogle' (google shopping)

# Asıl Kod Burada Çalışıyor Uzun Sürebilir

In [ ]:
Individual_EXACT_KEYWORD = list(zip(*[iter(KEYWORDS)]*1))
Individual_EXACT_KEYWORD = [list(x) for x in Individual_EXACT_KEYWORD]

dicti = {}
i = 1
for Country in COUNTRY:
    for keyword in Individual_EXACT_KEYWORD:
        
        pytrend.build_payload(kw_list=keyword, 
                              timeframe = DATE_INTERVAL, 
                              geo = Country, 
                              cat=CATEGORY,
                              gprop=SEARCH_TYPE)
        dicti[i] = pytrend.interest_over_time()
        
        i+=1
df_trends = pd.concat(dicti, axis=1)

In [ ]:
df_trends.columns = df_trends.columns.droplevel(0) 
df_trends.drop('isPartial',axis=1,inplace=True)
df_trends.reset_index(level=0,inplace=True)
df_trends

,date,Masa,Sandalye,Koltuk
0,2019-01-06,62,35,52
1,2019-01-13,58,37,58
2,2019-01-20,54,40,65
3,2019-01-27,54,37,64
4,2019-02-03,57,50,64
...,...,...,...,...
119,2021-04-18,55,60,70
120,2021-04-25,60,58,64
121,2021-05-02,55,54,57
122,2021-05-09,57,68,67


# Veri Setinin Görselleştirilmesi

Grafiklere zoom in/out yapabiliyorsunuz. Sağ üstteki tuşlarla düzenlenebiliyor.

In [ ]:
tt=df_trends.melt(id_vars=['date'], var_name='=')

In [ ]:
px.line(data_frame=tt,x='date',y='value',color='=')

In [ ]:
fig = px.bar(data_frame=tt,x='date',y='value',color='=')
fig.update_layout(barmode='group')

# Veriyi Excel'e Aktarma Kodu

Ekranın sol tarafındaki klasör simgesine bastığınızda Trends_Report dosyasını görebilirsiniz alttaki kodu çalıştırdıktan sonra

In [ ]:
df_trends.to_excel('Trends_Report.xlsx')

# İşinize Yarayabilecek Bazı İstatistiksel Yaklaşımlar

In [ ]:
df_trends['Month']=df_trends.date.apply(lambda x:x.month)
sum_t=df_trends.groupby('Month').sum()
mean_t=df_trends.groupby('Month').mean()
all_t=sum_t.merge(mean_t,how='left',on='Month',suffixes=['_sum','_mean'])
all_t

,Masa_sum,Sandalye_sum,Koltuk_sum,Masa_mean,Sandalye_mean,Koltuk_mean
Month,,,,,,
1,766,579,838,58.923077,44.538462,64.461538
2,726,594,809,60.500000,49.500000,67.416667
3,845,720,892,60.357143,51.428571,63.714286
4,815,719,765,67.916667,59.916667,63.750000
5,887,890,900,73.916667,74.166667,75.000000
6,667,657,692,74.111111,73.000000,76.888889
7,534,568,590,66.750000,71.000000,73.750000
8,585,560,607,65.000000,62.222222,67.444444
9,594,557,625,66.000000,61.888889,69.444444
